In [36]:
import os
import string
import random
from itertools import combinations, product

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [37]:
corpora_ru = list(open('WarAndPeace.txt', encoding="utf-8")) + list(open('AnnaKarenina.txt', encoding="utf-8"))

In [38]:
# переведем в строчные буквы, удалим все символы, кроме букв из алфавита
def clear_line(text, alphabet):
    line = ''
    for symbol in text.lower():
        if symbol in alphabet:
            line += symbol
    return ' '.join(line.split())
            
def clear_corpora(corpora, alphabet):
    cleaned = []
    for line in corpora:
        cleared_line = clear_line(line, alphabet)
        if len(cleared_line) > 0:
            cleared_line += ' '
            cleaned.append(cleared_line)
            
    return cleaned

alphabet_ru = 'абвгдеёжзийклмнопрстуфхцчшщъыьэюя '
corpora_ru = clear_corpora(corpora_ru, alphabet_ru)

1. Реализуйте базовый частотный метод по Шерлоку Холмсу:

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе совсем вряд ли сработает), 
- зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [39]:
def calculate_freques(alphabet, corpora_data):
    total_number = len(alphabet)
    freques = {''.join(symbol): 0 for symbol in alphabet}
    
    for line in corpora_data:
        for symbol in line:
            freques[symbol] += 1
            total_number += 1

    for symbol in freques:
        freques[symbol] /= total_number
    
    return freques

freques_ru = calculate_freques(alphabet_ru, corpora_ru)

In [40]:
# тестовый текст, на котором будем проверять алгоритмы
message ="""
Между тем Холмс, ненавидевший своей цыганской душой всякую форму светской жизни, оставался жить в нашей квартире на Бейкер-стрит, окруженный грудами своих старых книг, чередуя недели увлечения кокаином с приступами честолюбия, дремотное состояние наркомана – с дикой энергией, присущей его натуре.
"""
test_message = clear_line(message, alphabet_ru)
print(test_message)

между тем холмс ненавидевший своей цыганской душой всякую форму светской жизни оставался жить в нашей квартире на бейкерстрит окруженный грудами своих старых книг чередуя недели увлечения кокаином с приступами честолюбия дремотное состояние наркомана с дикой энергией присущей его натуре


In [41]:
class RandomPermuteCipher:
    """шифрователь случайной перестановкой символов"""
    def __init__(self, alphabet):
        self.alphabet = alphabet
        
        permutation = np.random.permutation(len(alphabet))
        self.cipher_dict = {}
        for i in range(len(alphabet)):
            self.cipher_dict[alphabet[i]] = alphabet[permutation[i]]

    def cipher(self, message):
        return ''.join([self.cipher_dict.get(symbol, '') for symbol in message])

In [42]:
# расшифровывает по готовому словарю
def decipher(ciphered, decipher_dict):
    return ''.join([decipher_dict.get(symbol, '') for symbol in ciphered])

In [43]:
cipher = RandomPermuteCipher(alphabet_ru)

ciphered = cipher.cipher(test_message)
print(ciphered)

укжтя ьку июруе чкчпйцткйвцъ ейюкъ нёмпчедюъ тявюъ йещдяг бюыуя ейкьедюъ жцачц юеьпйпрещ жцьф й чпвкъ дйпыьцык чп окъдкыеьыць юдыяжкччёъ мыятпуц ейюци еьпыёи дчцм лкыктящ чкткрц яйрклкчцщ дюдпцчюу е зыцеьязпуц лкеьюргоцщ тыкуюьчюк еюеьющчцк чпыдюупчп е тцдюъ счкымцкъ зыцеяшкъ кмю чпьяык


In [47]:
def decipher_unigramm(ciphered, freques_dict):
    """
    1. Подсчитываем частоты символов в зашифрованном сообщении.
    2. Упорядочиваем по убыванию частоты и составляем словарь расшифровки,
    сопоставляя зашифрованные символы символам из корпуса по частоте.
    """
    freques = {symbol : 0 for symbol in set(ciphered)}
    for symbol in ciphered:
        freques[symbol] += 1

    freques = sorted(
        [(letter, freq) for letter, freq in freques.items()], 
        key=lambda p : p[1], reverse=True
    )
    corpus_freques = sorted(
        [(letter, freq) for letter, freq in freques_dict.items()], 
        key=lambda p : p[1], reverse=True
    )

    decipher_dict = {}
    for i in range(len(freques)):
        decipher_dict[freques[i][0]] = corpus_freques[i][0]
    
    return decipher_dict

In [48]:
decipher_dict = decipher_unigramm(ciphered, freques_ru)
decipher(ciphered, decipher_dict)

'могур лом жаьмн иоисдеуодзев ндаов щчясинкав урзав днпкрш хатмр ндолнкав геюие анлсдсьнп гелф д исзов кдстлето ис йовкотнлтел актргоиичв ятрусме ндаеж нлстчж киея ыотоурп иоуоье рдьоыоиеп каксеиам н бтенлрбсме ыонлаьшйеп утомалиао нанлапиео исткамсис н уекав циотяеов бтенрэов ояа ислрто'

In [49]:
# доля правильно расшифрованных букв
def quality(cipher_dict, decipher_dict):
    n_correct = 0
    for symbol in cipher_dict.keys():
        ciphered_symbol = cipher_dict[symbol]
        if ciphered_symbol in decipher_dict and symbol == decipher_dict[ciphered_symbol]:
            n_correct += 1
    return n_correct / len(decipher_dict)
    
quality(cipher.cipher_dict, decipher_dict)

0.09375

Качество очень низкое

2. Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:

подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
проведите тестирование аналогично п.1, но при помощи биграмм.

In [50]:
def get_bigramm_freques(alphabet, corpora_data):
    total_number = len(alphabet) ** 2
    freques = {''.join(bigramm) : 1 for bigramm in product(alphabet, repeat=2)}
    
    for line in corpora_data:
        for idx in range(len(line) - 1):
            bigramm = line[idx:idx + 2]
            freques[bigramm] += 1
            total_number += 1
            
    for ngramm, freq in freques.items():
        freques[ngramm] = np.log(freq / total_number)
        
    return freques

bigramm_logfreques_ru = get_bigramm_freques(alphabet_ru, corpora_ru)

In [51]:
def get_decipher_bigramm(ciphered, freques_dict):
    text_symbols = list(set(ciphered))
    freques = {''.join(bigramm):1 for bigramm in product(text_symbols, repeat=2)}

    for idx in range(len(ciphered) - 1):
        bigramm = ciphered[idx:idx + 2]
        freques[bigramm] += 1

    freques = sorted(
        [(bigramm, freq) for bigramm, freq in freques.items()], 
        key=lambda p : p[1], reverse=True
    )
   
    corpus_freques = sorted(
        [(bigramm, freq) for bigramm, freq in freques_dict.items()], 
        key=lambda p : p[1], reverse=True
    )

    decipher_dict = {}
    
    for i in range(len(freques)):
        l1, l2 = freques[i][0]
        if l1 in decipher_dict:
            if l2 in decipher_dict:
                pass
            else:
                for j in range(len(corpus_freques)):
                    d1, d2 = corpus_freques[j][0]
                    if (d1 in decipher_dict.values() and 
                        d2 not in decipher_dict.values()):
                        decipher_dict[l2] = d2
                        break
        else:
            if l2 in decipher_dict:
                for j in range(len(corpus_freques)):
                    d1, d2 = corpus_freques[j][0]
                    if (d1 not in decipher_dict.values() and 
                        d2 in decipher_dict.values()):
                        decipher_dict[l1] = d1
                        break
            else:
                for j in range(len(corpus_freques)):
                    d1, d2 = corpus_freques[j][0]
                    if (d1 not in decipher_dict.values() and 
                        d2 not in decipher_dict.values()):
                        decipher_dict[l1] = d1
                        decipher_dict[l2] = d2
                        break
                
    return decipher_dict

decipher_dict = get_decipher_bigramm(ciphered, bigramm_logfreques_ru)

In [52]:
decipher(ciphered, decipher_dict)

'кийбы ник мрдкс аиалвебивэео сврио фючласпро быэро встпыш урякы свинспро йезае рснлвлдст йенж в алэио пвлянеяи ал щиопиясняен рпяыйиааюо чяыблке сврем снляюм паеч ьияибыт аибиде ывдиьиает прплеарк с гяесныглке ьиснрдшщет бяикрнари срснртаеи аляпрклал с бепро цаиячеио гяесыхио ичр алныяи'

Все еще очень плохо

In [53]:
decipherQuality(cipher.cipher_dict, decipher_dict)

0.125

3. Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:

- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились

Прежде всего, построим критерий качества получаемого после расшифровки текста на основе статистики биграмм (в более общем случае - n-грамм). Будем считать, что текст - марковская цепь. В качестве критерия качества будем использовать логарифм правдоподобия текста:$$llh = \sum \log\left(p(symbol_{i+n}|symbol_{i}, \dots symbol_{i + n - 1})\right)$$Здесь $\log\left(p(symbol_{i+n}|symbol_{i}, \dots symbol_{i + n - 1})\right)$ - логарифм условной вероятности появления символа при условии известных $(n - 1)$ предыдущих символов (для биграмм - при условии предыдущего символа). Эти вероятности будем оценивать на основе статистики n-грамм корпуса текстов.

In [54]:
def build_ngramm(alphabet, corpora_data, ngramm_length=2):
    total_number = len(alphabet) ** ngramm_length
    freques = {''.join(ngramm) : 1 for ngramm in product(alphabet, repeat=ngramm_length)}
    
    for line in corpora_data:
        for i in range(len(line) - ngramm_length + 1):
            ngramm = line[i : i + ngramm_length]
            freques[ngramm] += 1
            total_number += 1


    freques_cond = {''.join(ngramm) : {symbol : 0 for symbol in alphabet} 
                    for ngramm in product(alphabet, repeat=ngramm_length - 1)}
    tot_num = {''.join(ngramm) : 0 for ngramm in product(alphabet, repeat=ngramm_length - 1)}
    for ngramm, freq in freques.items():
        freques_cond[ngramm[:-1]][ngramm[-1]] += freq
        tot_num[ngramm[:-1]] += freq
    for prefix, freq_mar in freques_cond.items():
        for symbol, freq in freq_mar.items():
            freques_cond[prefix][symbol] = np.log(freq / tot_num[prefix])
            
    return freques_cond

bigramm_logfreques_ru = build_ngramm(alphabet_ru, corpora_ru, ngramm_length=2)

In [55]:
# логарифм правдоподобия текста по частотам биграмм
def calculate_bigramm_LLH(text):
    llh = bigramm_logfreques_ru[' '][text[0]]
    for i in range(len(text) - 1):
        llh += bigramm_logfreques_ru[text[i]][text[i + 1]]
    return llh


В качестве начального приближения возьмем декодирующую перестановку, полученную по частотам отдельных букв.

Используя MCMC-сэмплирование, построим последовательность дешифрующих перестановок символов, переставляя на каждом шаге пару символов. Считаем, что распределение дешифрующих перестановок пропорционально правдоподобию расшифрованного текста. Перестановку с лучшим правдоподобием сохраняем.

In [60]:
def get_decipher_MCMC(ciphered, LLH, unigramm_dict, steps=100):
    # начальное приближение - словарь по частотам отдельных букв
    decipher_dict = getDecipherUnigramm(ciphered, unigramm_dict)
    
    # дополним начальное приближение до полного алфавита
    for i, s in enumerate(unigramm_dict.keys() - decipher_dict.values()):
        decipher_dict[chr(ord(max(decipher_dict.keys())) + 1 + i)] = s
    
    max_llh = LLH(decipher(ciphered, decipher_dict))
    best_dict = decipher_dict.copy()
    
    prev_llh = max_llh

    symbols = list(decipher_dict.keys())
    symbols_pairs = list(combinations(symbols, 2))
    
    iteration = 0
    while iteration < steps:
        iteration += 1
        for a, b in np.random.permutation(symbols_pairs):
            new_dict = decipher_dict.copy()
            new_dict[a], new_dict[b] = new_dict[b], new_dict[a]
            llh = LLH(decipher(ciphered, new_dict))
            if llh > prev_llh or random.random() < np.exp(llh - prev_llh):
                decipher_dict = new_dict.copy()
                prev_llh = llh

            if llh > max_llh:
                max_llh = llh
                best_dict = new_dict.copy()
                iteration = 0
                break

    return best_dict

In [62]:
%%time
decipher_dict = get_decipher_MCMC(ciphered, calculate_bigramm_LLH, freques_ru)

CPU times: user 8.98 s, sys: 79.5 ms, total: 9.06 s
Wall time: 9.16 s


In [63]:
decipher(ciphered, decipher_dict)

'между тем холмс ненавидевший своей цыганской душой всякую форму светской жизни оставался жить в нашей квартире на бейкерстрит окруженный грудами своих старых книг чередуя недели увлечения кокаином с приступами честолюбия дремотное состояние наркомана с дикой энергией присущей его натуре'

In [64]:
decipherQuality(cipher.cipher_dict, decipher_dict)

0.9411764705882353

4. Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏


In [65]:
ciphered_4 = '←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏'

In [67]:
decipher(
    ciphered_4, 
    getDecipherMCMC(
        ciphered_4, 
        calculate_bigramm_LLH, 
        freques_ru,
        steps=1000
    )
)

'если вы вимите нордальный или почти нордальный текст у этого сообщения который легко прочитать скорее всего вы все смелали правильно и получите даксидальный балл за послемнее четвертое замание курса хотя конечно я ничего не обещаж'

5. Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

In [69]:
trigramm_logfreques_ru = build_ngramm(alphabet_ru, corpora_ru, ngramm_length=3)

In [70]:
def calculate_trigramm_LLH(text):
    llh = bigramm_logfreques_ru[' '][text[0]] + bigramm_logfreques_ru[text[0]][text[1]]
    for i in range(len(text) - 2):
        llh += trigramm_logfreques_ru[text[i:i + 2]][text[i + 2]]
    return llh

In [75]:
decipher(
    ciphered_4, 
    getDecipherMCMC(
        ciphered_4, 
        calculate_trigramm_LLH, 
        freques_ru,
        steps=1000
    )
)

'если вы видите нормальный или почти нормальный текст у этого сообщения который легко прочитать скорее всего вы все сделали правильно и получите максимальный балл за последнее четвертое задание курса хотя конечно я ничего не обещаю'